In [1]:
import pandas as pd
import folium
from ipywidgets import Dropdown, Output, VBox
from IPython.display import display, clear_output
from pathlib import Path

# ── CONFIGURATION ──
FILENAME = "Oslo.csv"

# ── LOAD DATA ──
df = pd.read_csv(
    Path(FILENAME),
    usecols=['date_time_utc', 'mmsi', 'latitude', 'longitude', 'ship_name'],
    parse_dates=['date_time_utc'],
    dayfirst=True,
    low_memory=False
)

# ── PREPARE VESSEL LIST ──
vessel_df = df[['mmsi', 'ship_name']].drop_duplicates().sort_values('ship_name')
options = [(f"{row.ship_name} ({row.mmsi})", row.mmsi) for _, row in vessel_df.iterrows()]

dropdown = Dropdown(options=options, description='Vessel:')
out = Output()

# ── PLOTTING FUNCTION ──
def plot_vessel(change):
    mmsi_selected = change['new']
    sub = df[df['mmsi'] == mmsi_selected].sort_values('date_time_utc')
    coords = list(zip(sub['latitude'], sub['longitude']))
    if not coords:
        with out:
            clear_output(wait=True)
            print("No data for this vessel.")
        return

    # center map on midpoint of journey
    mid_idx = len(coords) // 2
    center = coords[mid_idx]

    m = folium.Map(location=center, zoom_start=10)
    # journey line
    folium.PolyLine(coords, color='blue', weight=2.5, opacity=0.8).add_to(m)
    # start marker
    start_ts = sub['date_time_utc'].iloc[0]
    folium.Marker(
        location=coords[0],
        icon=folium.Icon(color='green', icon='play'),
        popup=f"Start: {start_ts}"
    ).add_to(m)
    # end marker
    end_ts = sub['date_time_utc'].iloc[-1]
    folium.Marker(
        location=coords[-1],
        icon=folium.Icon(color='red', icon='stop'),
        popup=f"End: {end_ts}"
    ).add_to(m)

    with out:
        clear_output(wait=True)
        display(m)

# ── WIRE UP INTERACTION ──
dropdown.observe(plot_vessel, names='value')

# ── DISPLAY WIDGET ──
display(VBox([dropdown, out]))

# ── INITIAL PLOT ──
plot_vessel({'new': options[0][1]})


In [3]:
import pandas as pd
import folium
import random
from pathlib import Path

# ── CONFIGURATION ──
FILENAME = "Oslo.csv"

# ── LOAD DATA ──
df = pd.read_csv(
    Path(FILENAME),
    usecols=['date_time_utc', 'mmsi', 'latitude', 'longitude'],
    parse_dates=['date_time_utc'],
    low_memory=False
)

# ── PREPARE MAP CENTER ──
# compute overall center
mean_lat = df['latitude'].mean()
mean_lon = df['longitude'].mean()
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=10)

# ── PLOT EACH VESSEL PATH WITH UNIQUE COLOR ──
for mmsi, group in df.groupby('mmsi'):
    # skip if too few points
    pts = list(zip(group['latitude'], group['longitude']))
    if len(pts) < 2:
        continue
    # generate a random color in hex
    color = "#%06x" % random.randint(0, 0xFFFFFF)
    # add polyline
    folium.PolyLine(
        pts,
        color=color,
        weight=2,
        opacity=0.7,
        tooltip=str(mmsi)
    ).add_to(m)

# ── DISPLAY MAP ──
m